In [1]:
import os
import pybullet as p
import open3d as o3d
import numpy as np
import copy
p.connect(p.GUI)

pybullet build time: May 20 2022 19:44:17


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 2080/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 510.85.02
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 510.85.02
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 2080/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NV

0

ven = NVIDIA Corporation


In [2]:
def create_handle(alpha=0.8):
    floor_id = p.loadURDF("../envs/assets/plane.urdf", useFixedBase=True)
    init_xyz = np.array([0.145,0.0,0.0])
    shell_id = p.loadURDF("../envs/assets/handle_shell.urdf",basePosition=init_xyz,useFixedBase=True)
    init_xyz = np.array([0.1,0.0,0.0])
    o_id = p.loadURDF("../envs/assets/handle.urdf",basePosition=init_xyz,useFixedBase=True)
    return o_id

# For deocclusion
aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound=np.array([-100, -0.1, 0.005]),
                                                        max_bound=np.array([0.095, 0.1, 100]))


In [3]:
o_id = create_handle()
mesh = o3d.io.read_triangle_mesh("../envs/assets/handle.obj")
mesh.compute_triangle_normals()
mesh.compute_vertex_normals()
pcd = mesh.sample_points_poisson_disk(2048, use_triangle_normal=True)
sp = o3d.geometry.TriangleMesh.create_sphere(0.01)

In [4]:
o3d.visualization.draw_geometries([pcd, sp])

In [5]:
theta = -np.pi/2
rot_mat = np.array([[np.cos(theta), 0, np.sin(theta)],
                    [0, 1, 0],
                    [-np.sin(theta), 0, np.cos(theta)]])

In [6]:
pcd.rotate(rot_mat, center=[0,0,0])

PointCloud with 2048 points.

In [7]:
cropped_pcd = copy.deepcopy(pcd).crop(aabb)
cropped_pcd.rotate(rot_mat.T, center=[0,0,0])

PointCloud with 1545 points.

In [8]:
o3d.visualization.draw_geometries([cropped_pcd, sp])

In [9]:
o3d.io.write_point_cloud("pose_10_pcd.ply", cropped_pcd)

True

In [11]:
o3d.visualization.draw_geometries_with_vertex_selection([cropped_pcd])

[Open3D INFO] Clearing all points from selection.
[Open3D INFO] Adding point #497 (0.05, -0.01, 0.02) to selection.
[Open3D INFO] Adding point #1252 (0.10, 0.05, 0.02) to selection.
[Open3D INFO] Adding point #1371 (0.10, 0.03, 0.02) to selection.
[Open3D INFO] Removing point #1371 from selection.
[Open3D INFO] Adding point #1322 (0.10, 0.02, 0.02) to selection.
[Open3D INFO] Adding point #1269 (0.10, -0.00, 0.02) to selection.
[Open3D INFO] Adding point #1488 (0.10, -0.04, 0.02) to selection.


In [14]:
points = np.asarray(cropped_pcd.points)
normals = np.asarray(cropped_pcd.normals)
tip_pos = np.array([points[497], points[1252], points[1322], points[1269], points[1488]])
normals = np.array([normals[497], normals[1252], normals[1322], normals[1269], normals[1488]])
grasp = np.hstack([tip_pos, normals])

In [15]:
sps = []
for i in range(5):
    sp = o3d.geometry.TriangleMesh.create_sphere(0.01)
    sp.translate(grasp[i,:3])
    sps.append(sp)

In [16]:
o3d.visualization.draw_geometries([cropped_pcd]+sps)

In [17]:
np.save("pose_10.npy", grasp)

: 